## 3. Implement linear regression from scratch

If you want to dive deeper to understand linear regression you can implement it yourself following [these exercises here](https://github.com/kokchun/Machine-learning-AI22/blob/main/Exercises/E00_linear_regression.ipynb).